# Company database quickstart\n\nThis notebook shows how to connect to the practice PostgreSQL database with SQLAlchemy and run a query with pandas. The example uses the company-style schema described in `docs/schema.md`.\n\n## Prerequisites\n\nInstall the Python dependencies (adjust for your environment):\n\n```bash\npython -m pip install pandas sqlalchemy psycopg2-binary\n```\n\nEnsure the database is running (for local development, `docker compose up -d`). Then set the connection values to match the `.env` or defaults from `docker-compose.yml`.

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine

db_user = os.getenv("POSTGRES_USER", "practice_user")
db_password = os.getenv("POSTGRES_PASSWORD", "practice_password")
db_name = os.getenv("POSTGRES_DB", "practice_db")
db_host = os.getenv("POSTGRES_HOST", "localhost")
db_port = os.getenv("POSTGRES_PORT", "5432")

engine = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)

engine

## Example: employees by department\n\nThis query joins employees to departments and shows the most recent hires.

In [ ]:
query = """
SELECT
  e.employee_id,
  e.first_name,
  e.last_name,
  d.name AS department,
  e.hire_date
FROM employee AS e
JOIN department AS d ON d.department_id = e.department_id
ORDER BY e.hire_date DESC
LIMIT 10;
"""

pd.read_sql(query, engine)

## Example: customer orders with totals\n\nAggregate order totals per customer to see top accounts.

In [ ]:
query = """
SELECT
  c.customer_id,
  c.name,
  COUNT(o.order_id) AS orders,
  SUM(o.total) AS lifetime_total
FROM customer AS c
LEFT JOIN sales_order AS o ON o.customer_id = c.customer_id
GROUP BY c.customer_id, c.name
ORDER BY lifetime_total DESC NULLS LAST
LIMIT 10;
"""

pd.read_sql(query, engine)

## Next steps\n\n- Use the tables in `docs/schema.md` to explore more joins (products, prices, order items).\n- Swap `pd.read_sql` for `pd.read_sql_query` or `pd.read_sql_table` depending on your workflow.\n- For production deployments, load credentials from a secrets manager instead of environment variables.